## Ejemplo de Sqoop

En este ejemplo vamos a configurar Sqoop para que realice una serie de importaciones desde una base de datos relacional. 

Se trata de una base de datos MySQL llamada "retail_db" que se encuentra instalada en el contenedor de Cloudera. Esta base de datos tiene las siguientes tablas:

- categories
- departments
- products
- order_items
- orders
- customers

<img src="files/sqoop-bd.png">

In [1]:
! mkdir -p sqoop
import os
os.chdir("sqoop")
! pwd

/media/notebooks/sqoop


## Primer ejemplo: 

Importamos una tabla, la tabla departments, codificando los datos con Parquet, comprimido con el algoritmo snappy, y creando una tabla en Hive llamada depts. La dejaremos en la base de datos de Hive llamada pruebasqoop, que crearemos ahora.

In [2]:
! hadoop fs -mkdir /user/cloudera/pruebasqoopB

In [3]:
%%writefile ejemplo
create database if not exists pruebasqoopB
Comment 'BD para pruebas de sqoop'
Location '/user/cloudera/pruebasqoopB';

describe database extended pruebasqoopB;

show databases;

Writing ejemplo


In [4]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f ejemplo

2019-05-14 18:18:35,412 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
opBjdbc:hive2://localhost:10000/default> create database if not exists pruebasqo 
0: jdbc:hive2://localhost:10000/default> Comment 'BD para pruebas de sqoop'
0: jdbc:hive2://localhost:10000/default> Location '/user/cloudera/pruebasqoopB'; 
INFO  : Compiling command(queryId=hive_20190514181818_58299d95-387f-467d-b029-627950a51098): create database if not exists pruebasqoopB
Comment 'BD para pruebas de sqoop'
Location '/user/cloudera/pruebasqoopB'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_2019051

In [5]:
! sqoop help

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:21:11 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
usage: sqoop COMMAND [ARGS]

Available commands:
  codegen            Generate code to interact with database records
  create-hive-table  Import a table definition into Hive
  eval               Evaluate a SQL statement and display the results
  export             Export an HDFS directory to a database table
  help               List available commands
  import             Import a table from a database to HDFS
  import-all-tables  Import tables from a database to HDFS
  import-mainframe   Import datasets from a mainframe server to HDFS
  job                Work with saved jobs
  list-databases     List available databases on a server
  list-tables        List available tables in a database
  merge              Merge results of incremental imports
  metastore          Run a standalone Sqoop metastore
  version            Display version informa

In [6]:
! sqoop help import

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:21:20 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
usage: sqoop import [GENERIC-ARGS] [TOOL-ARGS]

Common arguments:
   --connect <jdbc-uri>                         Specify JDBC connect
                                                string
   --connection-manager <class-name>            Specify connection manager
                                                class name
   --connection-param-file <properties-file>    Specify connection
                                                parameters file
   --driver <class-name>                        Manually specify JDBC
                                                driver class to use
   --hadoop-home <hdir>                         Override
                                                $HADOOP_MAPRED_HOME_ARG
   --hadoop-mapred-home <dir>                   Override
                                                $HADOOP_MAPRED_HOME_ARG
   --h

In [7]:
%%writefile ejemplo
sqoop import \
    --connect jdbc:mysql://quickstart:3306/retail_db \
    --username=retail_dba \
    --password=cloudera \
    --table departments \
    --compression-codec=snappy \
    --as-parquetfile \
    --hive-import \
    --hive-database pruebasqoopB \
    --hive-table depts \
    --hive-overwrite;

Overwriting ejemplo


In [8]:
! sh ./ejemplo

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:21:41 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
19/05/14 18:21:41 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/05/14 18:21:41 INFO tool.BaseSqoopTool: Using Hive-specific delimiters for output. You can override
19/05/14 18:21:41 INFO tool.BaseSqoopTool: delimiters with --fields-terminated-by, etc.
19/05/14 18:21:42 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/05/14 18:21:42 INFO tool.CodeGenTool: Beginning code generation
19/05/14 18:21:42 INFO tool.CodeGenTool: Will generate java class as codegen_departments
19/05/14 18:21:42 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `departments` AS t LIMIT 1
19/05/14 18:21:42 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `departments` AS t LIMIT 1
19/05/14 18:21:42 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /us

In [9]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoopB" -e "show tables;"

2019-05-14 18:23:33,055 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoopB
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514182323_730ddd56-b110-487b-9122-493c5024d304): show tables
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514182323_730ddd56-b110-487b-9122-493c5024d304); Time taken: 0.017 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514182323_730ddd56-b110-487b-9122-493c5024d304): show tables
INFO  : Starting task [Stage-0:D

In [10]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoopB" -e "select * from depts;"

2019-05-14 18:23:42,833 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoopB
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514182323_ff09f2cd-519e-42d0-96ae-6cebd114a883): select * from depts
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:depts.department_id, type:int, comment:null), FieldSchema(name:depts.department_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514182323_ff09f2cd-519e-42d0-96ae-6cebd114a883); Time taken: 0.392 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514182323_ff09f2cd-519

In [11]:
! hadoop fs -ls /user/cloudera/pruebasqoopB

Found 1 items
drwxr-xr-x   - root cloudera          0 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts


In [12]:
! hadoop fs -ls /user/cloudera/pruebasqoopB/depts

Found 6 items
drwxr-xr-x   - root cloudera            0 2019-05-14 18:21 /user/cloudera/pruebasqoopB/depts/.metadata
drwxr-xr-x   - root cloudera            0 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts/.signals
-rw-r--r--   1 root supergroup        745 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts/957b7c9e-c728-46c1-b6e4-8764ecea610d.parquet
-rw-r--r--   1 root supergroup        742 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts/9b20ed1e-19bb-4b59-b9c7-7582407aad95.parquet
-rw-r--r--   1 root supergroup        722 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts/9c3fa633-c337-4a6e-8f2f-4161e4797075.parquet
-rw-r--r--   1 root supergroup        707 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts/e27ef0e9-7c20-4b55-96e0-749c8b41dd3c.parquet


## Segundo ejemplo: 

Importamos solamente aquellas de las filas de la tabla departments que tengan como department_name el valor Fitness

In [17]:
%%writefile ejemplo
sqoop import \
    --connect jdbc:mysql://quickstart:3306/retail_db \
    --username=retail_dba \
    --password=cloudera \
    --table departments \
    --compression-codec=snappy \
    --as-parquetfile \
    --hive-import \
    --hive-database pruebasqoopB \
    --hive-table deptFitness \
    --where "department_name = 'Fitness'";

Overwriting ejemplo


In [18]:
! sh ./ejemplo

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:26:38 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
19/05/14 18:26:38 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/05/14 18:26:38 INFO tool.BaseSqoopTool: Using Hive-specific delimiters for output. You can override
19/05/14 18:26:38 INFO tool.BaseSqoopTool: delimiters with --fields-terminated-by, etc.
19/05/14 18:26:39 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/05/14 18:26:39 INFO tool.CodeGenTool: Beginning code generation
19/05/14 18:26:39 INFO tool.CodeGenTool: Will generate java class as codegen_departments
19/05/14 18:26:39 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `departments` AS t LIMIT 1
19/05/14 18:26:39 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `departments` AS t LIMIT 1
19/05/14 18:26:39 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /us

In [20]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoopB" -e "show tables;"

2019-05-14 18:27:44,909 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoopB
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514182727_341ec898-c9b7-4b9c-8178-3f0db34e54a3): show tables
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514182727_341ec898-c9b7-4b9c-8178-3f0db34e54a3); Time taken: 0.007 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514182727_341ec898-c9b7-4b9c-8178-3f0db34e54a3): show tables
INFO  : Starting task [Stage-0:D

In [21]:
! hadoop fs -ls /user/cloudera/pruebasqoopB/

Found 2 items
drwxr-xr-x   - root cloudera          0 2019-05-14 18:27 /user/cloudera/pruebasqoopB/deptfitness
drwxr-xr-x   - root cloudera          0 2019-05-14 18:22 /user/cloudera/pruebasqoopB/depts


In [22]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoopB" -e "select * from  deptfitness;"

2019-05-14 18:28:13,266 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoopB
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514182828_ad9691cb-23c6-4745-afd6-0ed405011333): select * from  deptfitness
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:deptfitness.department_id, type:int, comment:null), FieldSchema(name:deptfitness.department_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514182828_ad9691cb-23c6-4745-afd6-0ed405011333); Time taken: 0.075 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514

In [23]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoopB" -e "describe  deptfitness;"

2019-05-14 18:28:43,778 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoopB
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514182828_200a6059-8971-4fe3-88df-850546fa6058): describe  deptfitness
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:col_name, type:string, comment:from deserializer), FieldSchema(name:data_type, type:string, comment:from deserializer), FieldSchema(name:comment, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514182828_200a6059-8971-4fe3-88df-850546fa6058); Time taken: 0.074 seconds
INFO  : Concurrency mode is disabled, not creating a lock

## Ejercicio propuesto

1. Realiza la importación de la base de datos retail_db entera, incluyendo todas las tablas. El destino de esta base de datos debe ser la carpeta /user/cloudera/retailentera (que hay que crear antes de nada). Los datos deben estar codificados en avro, sin compresión. 

2. Crea una tabla externa Hive al menos para una de las tablas de la base de datos. 

3. Crea una segunda tabla externa en Hive, en la que insertaremos el resultado de una consulta select que realizaremos sobre la tabla externa Hive creada en el punto anterior. La consulta select puede ser cualquiera (selecciona los items cuyo id sea 1, por ejemplo).

4. Crea una nueva tabla en la base de datos relacional.

5. Exporta la tabla Hive creada en el punto 3 en la tabla relacional creada en el punto 4. 


-------------

Para el punto 4, puedes utilizar estas órdenes desde un terminal (es posible utilizar una tabla distinta):

Conéctate a tu contenedor con esta orden:

docker exec -ti IDCONTENEDOR  /bin/bash

Y luego emite estas órdenes (elimina lo que haytras las -->):

mysql -u retail_dba -p         
--> nos conectamos a mysql, utilizando el usuario retail_dba. Tras ejecutar esta orden deberemos introducir la clave, que es cloudera

use retail_db;
--> indicamos que deseamos utilizar la base de datos retail_db


CREATE TABLE depsB (
    department_id int,
    department_name varchar(45),
    PRIMARY KEY (department_id)
);
--> Creamos una tabla, que será donde dejaremos los datos exportados desde HDFS.



In [24]:
! hadoop fs -mkdir /user/cloudera/retailentera

In [25]:
%%writefile ejemplo
# TU CODIGO PARA EL PUNTO 1 VA AQUI


Overwriting ejemplo


In [26]:
! sh ./ejemplo

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:44:56 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
19/05/14 18:44:56 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/05/14 18:44:57 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/05/14 18:44:57 INFO tool.CodeGenTool: Beginning code generation
19/05/14 18:44:57 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `categories` AS t LIMIT 1
19/05/14 18:44:57 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `categories` AS t LIMIT 1
19/05/14 18:44:57 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /usr/lib/hadoop-mapreduce
Note: /tmp/sqoop-root/compile/3c9b40cd774583d996dadcb380660399/categories.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
19/05/14 18:44:58 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-root/compile/3c

In [27]:
! hadoop fs -ls /user/cloudera/retailentera/

Found 6 items
drwxr-xr-x   - root cloudera          0 2019-05-14 18:45 /user/cloudera/retailentera/categories
drwxr-xr-x   - root cloudera          0 2019-05-14 18:45 /user/cloudera/retailentera/customers
drwxr-xr-x   - root cloudera          0 2019-05-14 18:45 /user/cloudera/retailentera/departments
drwxr-xr-x   - root cloudera          0 2019-05-14 18:46 /user/cloudera/retailentera/order_items
drwxr-xr-x   - root cloudera          0 2019-05-14 18:46 /user/cloudera/retailentera/orders
drwxr-xr-x   - root cloudera          0 2019-05-14 18:46 /user/cloudera/retailentera/products


In [28]:
%%writefile ejemplo
create external table departmentssqoopavro (
  department_id  int,   
  department_name  STRING) 
STORED AS AVRO
LOCATION '/user/cloudera/retailentera/departments';

Overwriting ejemplo


In [29]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoop" -f ejemplo

2019-05-14 18:48:19,136 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoop
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
oopavro (ive2://localhost:10000/pruebasqoop> create external table departmentssq 
0: jdbc:hive2://localhost:10000/pruebasqoop>   department_id  int,   
0: jdbc:hive2://localhost:10000/pruebasqoop>   department_name  STRING) 
0: jdbc:hive2://localhost:10000/pruebasqoop> STORED AS AVRO
ra/departments';localhost:10000/pruebasqoop> LOCATION '/user/cloudera/retailente 
INFO  : Compiling command(queryId=hive_20190514184848_bb5cf35f-86e6-4eca-8c57-1697c47b8bce): create external table departmentssqoopavro (
  department_id  int,   
  department_name  STRING) 
STORED AS AVRO
LOCATION '/user/cloudera/retailentera/d

In [30]:
%%writefile ejemplo
create external table departmentssqoopavroB (
  department_id  int,   
  department_name  STRING) 
STORED AS AVRO
LOCATION '/user/cloudera/retailentera/departmentsB';

Overwriting ejemplo


In [31]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoop" -f ejemplo

2019-05-14 18:48:51,643 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoop
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
oopavroBhive2://localhost:10000/pruebasqoop> create external table departmentssq  (
0: jdbc:hive2://localhost:10000/pruebasqoop>   department_id  int,   
0: jdbc:hive2://localhost:10000/pruebasqoop>   department_name  STRING) 
0: jdbc:hive2://localhost:10000/pruebasqoop> STORED AS AVRO
ra/departmentsB';ocalhost:10000/pruebasqoop> LOCATION '/user/cloudera/retailente 
INFO  : Compiling command(queryId=hive_20190514184848_17462f31-7223-41fc-9cbc-8f91af1c314a): create external table departmentssqoopavroB (
  department_id  int,   
  department_name  STRING) 
STORED AS AVRO
LOCATION '/user/cloudera/retailenter

In [32]:
! beeline -u "jdbc:hive2://localhost:10000/pruebasqoop" -e "INSERT OVERWRITE TABLE departmentssqoopavroB SELECT * FROM departmentssqoopavro WHERE department_id = 3;"

2019-05-14 18:49:47,865 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/pruebasqoop
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514184949_a006d629-e88f-410b-8204-5dfc4f9dd05c): INSERT OVERWRITE TABLE departmentssqoopavroB SELECT * FROM departmentssqoopavro WHERE department_id = 3
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:departmentssqoopavro.department_id, type:int, comment:null), FieldSchema(name:departmentssqoopavro.department_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514184949_a006d629-e88f-410b-8204-5dfc4f9dd05c); Time taken: 0.279 seconds
INFO  : Concurrency

Ahora ejecutamos en el mysql la orden para crear la nueva tabla
Esto no se muestra aquí. Tras eso ...



In [33]:
%%writefile ejemplo
# TU CODIGO PARA EL PUNTO 5 VA AQUI

Overwriting ejemplo


In [34]:
! sh ./ejemplo

Please set $ACCUMULO_HOME to the root of your Accumulo installation.
19/05/14 18:58:46 INFO sqoop.Sqoop: Running Sqoop version: 1.4.6-cdh5.7.0
19/05/14 18:58:46 WARN tool.BaseSqoopTool: Setting your password on the command-line is insecure. Consider using -P instead.
19/05/14 18:58:46 INFO manager.MySQLManager: Preparing to use a MySQL streaming resultset.
19/05/14 18:58:46 INFO tool.CodeGenTool: Beginning code generation
19/05/14 18:58:47 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `depsB` AS t LIMIT 1
19/05/14 18:58:47 INFO manager.SqlManager: Executing SQL statement: SELECT t.* FROM `depsB` AS t LIMIT 1
19/05/14 18:58:47 INFO orm.CompilationManager: HADOOP_MAPRED_HOME is /usr/lib/hadoop-mapreduce
Note: /tmp/sqoop-root/compile/ada9d4763358bc6acf734358eb0f543b/depsB.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
19/05/14 18:58:48 INFO orm.CompilationManager: Writing jar file: /tmp/sqoop-root/compile/ada9d4763358bc6ac

Para comprobar que la exportación se ha realizado con éxito, ejecuta la siguiente orden, donde depsB es la tabla que hemos creado en mysql. 

In [35]:
! mysql -u retail_dba -pcloudera -e "use retail_db; select * from depsB";

+---------------+-----------------+
| department_id | department_name |
+---------------+-----------------+
|             3 | Footwear        |
+---------------+-----------------+
